# Sentiment Analysis Of Songs across decades

I analyzed different decades starting from 1950’s to see what decades had the most “positive” sounding lyrics. I created a list of top 10 songs from each decade in a CSV file and fed it into an API to grab each song’s lyrics and turned that into a dataframe. After this, I ran Google Cloud’s sentiment analysis algorithm from the Natural Language API provided by Google which gave us a sentiment score along with the magnitude which represented the content amount. With this newly added information in the dataframe, I was able to do some analysis and see what decades had the most positive sounding songs. 



In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
%pylab inline
import pandas as pd
import sys
import subprocess

Populating the interactive namespace from numpy and matplotlib


## Download packages and setup


In [54]:
!pip install google-cloud-language

In [53]:

!pip install --upgrade google-cloud-natural-language
!pip install -t lib/ --upgrade google-api-python-client
!pip install -t lib/ --upgrade google-cloud
!pip freeze | grep google-cloud-language

Requirement already up-to-date: google-cloud-natural-language in /usr/local/lib/python3.6/dist-packages (0.24.0)
  Using cached https://files.pythonhosted.org/packages/63/e1/f465e0e8138cc76587d5b626c16c649b72f41a5cdbf414f8959221e03096/google_api_python_client-1.9.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bd/4e/992849016f8b0c27fb604aafd0a7a724db16128906197bd1245c6f18e6a1/google_auth_httplib2-0.0.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/48/76/bd5e3ebd571b9f688ae123a2bd84b2dc9f75e1214a83b8a759bc16e2fc35/google_api_core-1.21.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/09/4b/a25adc25200de0f558749908ffa78eb8d4b8750775c9c944ffc0dc2e661c/google_auth-1.19.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b3/ad/d9d9331850ea5bd4f5cb8c650c0bfa119a4abd6b0ad7c45b6506bc979fc0/httplib2-0.18.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ee/ff/48b

  Using cached https://files.pythonhosted.org/packages/ba/b1/7c54d1950e7808df06642274e677dbcedba57f75307adf2e5ad8d39e5e0e/google_cloud-0.34.0-py2.py3-none-any.whl
google-cloud-language==0.24.0


In [3]:
!pip install requests

In [4]:
## import the os package
import os
## assign the OS environment variable
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='/content/gdrive/My Drive/GCP Service Key/QAC Project-736f77fc4a4f.json'

## read the value to confirm the change
os.environ['GOOGLE_APPLICATION_CREDENTIALS']

'/content/gdrive/My Drive/GCP Service Key/QAC Project-736f77fc4a4f.json'

## Import CSV File

In [5]:
songsFromDecades = pd.read_csv("/content/gdrive/My Drive/Qac239Final/export_dataframeWithSentiment.csv")

In [6]:
n = songsFromDecades.shape[0]

In [7]:
songsFromDecades['lyrics'] = ""
songsFromDecades['result'] = ""


In [8]:
# files = os.listdir("/content/gdrive/My Drive/Qac239Final/videos")
df = pd.DataFrame(songsFromDecades)
# df['full'] = "/content/gdrive/My Drive/Qac239Final/videos/" + df.file_name

In [9]:
df

,songs,artist,decade,lyrics,curse_word_count,sentiment_analysis,sentiment_analysis_score,sentiment_analysis_magnitude,result
0,Come Fly With Me,Frank Sinatra,1950,,NaN,NaN,NaN,NaN,
1,That's Amore,Dean Martin,1950,,NaN,NaN,NaN,NaN,
2,Jailhouse Rock,Elvis Presley,1950,,NaN,NaN,NaN,NaN,
3,I Walk The Line,Johnny Cash,1950,,NaN,NaN,NaN,NaN,
4,I've Got You Under My Skin,Frank Sinatra,1950,,NaN,NaN,NaN,NaN,
...,...,...,...,...,...,...,...,...,...
65,Shake It Off,Taylor Swift,2010,,NaN,NaN,NaN,NaN,
66,Thinking Out Loud,Ed Sheeran,2010,,NaN,NaN,NaN,NaN,
67,Marry You,Bruno Mars,2010,,NaN,NaN,NaN,NaN,
68,Blurred Lines,Robin Thicke Featuring T.I. & Pharrell,2010,,NaN,NaN,NaN,NaN,


## Setup Mourits-Lyrics service to get lyrics of each song.
###Service Key is hidden for security reasons. Please head to https://rapidapi.com/PlanetTeamSpeak/api/mourits-lyrics to create your own API Key! 


Use Music api to get all songs
https://api.musixmatch.com/ws/1.1/matcher.lyrics.get?format=jsonp&callback=callback&q_track=HIGHEST%20IN%20THE%20ROOM&apikey=06d2c7b479fcf9517d2ec55303f6e863


In [14]:
apiKey = '[Enter API KEY Here]'
URL = 'https://mourits-lyrics.p.rapidapi.com/?'
headers = {'x-rapidapi-key': apiKey}

## Make requests to service and add to the dataframe

In [31]:
import time
for i in range(0,n): 
  PARAMS = {'a': df.artist[i], 's': df.songs[i]} 
  r = requests.get(url = URL, headers = headers, params = PARAMS) 
  roles =  r.json()
  roles['result']['lyrics']
  if roles['success'] == True:
      df.lyrics[i] = roles['result']['lyrics'] #Extract lyrics
  time.sleep(3)  #Do this every 3 seconds to avoid time outs. 

{'message': 'You are not subscribed to this API.'}
{'message': 'You are not subscribed to this API.'}
{'message': 'You are not subscribed to this API.'}
{'message': 'You are not subscribed to this API.'}


KeyboardInterrupt: ignored

In [10]:
exportedDataframe = pd.read_csv("/content/gdrive/My Drive/Qac239Final/export_dataframe.csv")

In [11]:
exportedDataframe['sentiment_analysis_score'] = ""
exportedDataframe['sentiment_analysis_magnitude'] = ""

In [12]:
exporteddf = exportedDataframe

In [13]:
lengthOfFrame = len(exportedDataframe)


## Do sentiment analysis using the songs as an input and update CSV 

In [15]:
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

# Instantiates a client
client = language.LanguageServiceClient()

# The text to analyze

for i in range(lengthOfFrame):
  text = exporteddf.lyrics[i]
  if str(text) != 'nan':
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)

    # Detects the sentiment of the text
    sentiment = client.analyze_sentiment(document=document).document_sentiment

    print('Text: {}'.format(text))
    print('Sentiment: {}, {}'.format(sentiment.score, sentiment.magnitude))
    #Update sentiments inside dataframe
    exporteddf.sentiment_analysis_score[i] = sentiment.score
    exporteddf.sentiment_analysis_magnitude[i] = sentiment.magnitude



ImportError: ignored

## Save CSV File with Sentiment Analysis

In [16]:
exporteddf.to_csv(r'/content/gdrive/My Drive/Qac239Final/SentimentAnalysisOfSongs.csv', index = None, header = True)

The 1960s had the most positive songs with the top 10 songs achieving an average Sentimental Analysis of ~0.5, far above the rest of its competition.